# Imports

In [25]:
import math
import pandas as pd
import inflection

# Helper Functions

# Loading Data

In [5]:
df_sales_raw = pd.read_csv('dataset/train.csv', low_memory=False)
df_store_raw = pd.read_csv('dataset/store.csv', low_memory=False)

In [8]:
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

# Data description

## Rename Columns

In [11]:
df1 = df_raw.copy()
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [13]:
old_cols = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
               'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType',
               'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
               'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
               'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

new_cols = list( map( snakecase, old_cols ) )

df1.columns = new_cols

In [14]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## Data Dimension

In [15]:
df1.shape

(1017209, 18)

## Data types

In [20]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## Check NAs

In [24]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## Fillout NA

In [35]:
# competition_distance

df1['competition_distance'] = (df1['competition_distance']
                               .apply(lambda x: 2000000.0 if math.isnan(x) 
                                      else x))


# competition_open_since_month

df1['competition_open_since_month'] = (df1.apply(
    lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) 
    else x['competition_open_since_month'], axis=1))

# competition_open_since_year

df1['competition_open_since_year'] = (df1.apply(
    lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) 
    else x['competition_open_since_year'], axis=1))
                   
# promo2_since_week

df1['promo2_since_week'] = (df1.apply(
    lambda x: x['date'].week if math.isnan(x['promo2_since_week']) 
    else x['promo2_since_week'], axis=1))


# promo2_since_year

df1['promo2_since_year'] = (df1.apply(
    lambda x: x['date'].year if math.isnan(x['promo2_since_year']) 
    else x['promo2_since_year'], axis=1))

# promo_interval

month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna( 0 )

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = (df1[['promo_interval', 'month_map']]
                   .apply(lambda x: 0 if x['promo_interval'] == 0 else 1 
                          if x['month_map'] in x['promo_interval']
                          .split(',') else 0, axis=1))

In [36]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

In [40]:
df1.sample(5).T

,873158,435276,711027,56663,220939
store,899,97,443,914,170
day_of_week,5,5,3,4,3
date,2013-05-10 00:00:00,2014-06-06 00:00:00,2013-10-02 00:00:00,2015-06-11 00:00:00,2015-01-14 00:00:00
sales,5431,8336,5162,8166,4784
customers,630,806,556,1095,480
open,1,1,1,1,1
promo,0,1,0,0,1
state_holiday,0,0,0,0,0
school_holiday,1,0,0,0,0
store_type,d,d,d,c,a
